In [6]:
from codes_by import *
import socket
from scapy.all import *
from termcolor import colored, cprint

class TS():
    def __init__(self, pkt):
        self.init = True
        if 'Raw' in pkt:
#             self.data = pkt[Raw].load
            self.data = b''
            self.nseq = pkt[TCP].seq + len(self.data)
        else:
            self.data = b''
            if pkt[TCP].flags & 2 == 2:
                self.nseq = pkt[TCP].seq  + 1
    def add_packet(self, pkt):
        print('&&&&&&&&&&&&&&&&', pkt.time)
        ret = []
        if pkt[TCP].seq != self.nseq:
#             print(pkt[TCP].seq, self.nseq)
#             print('Oops')
            return ret
        if 'Raw' in pkt:
            self.data += pkt[Raw].load
            self.nseq = pkt[TCP].seq + len(pkt[Raw].load)
            if self.init:
                head = self.data.find(b'\r\n\r\n')
                if head > 0:
                    begin = head+4
                    self.init = False
                    self.c = begin
            if not self.init:
                for i in range(5):
                    found_frame = False
                    if len(self.data) - self.c >= 2:
                        ws_op, ws_len = struct.unpack('BB', self.data[self.c:self.c+2])
                        ll = 0
                        mm = 0
                        mask_flag = ws_len & 0x80
                        ws_len = ws_len & 0x7f
                        if mask_flag > 0: mm = 4
                        if ws_len == 126:
                            ll = 2
                            if len(self.data) - self.c >= 2+ll+4:
                                ws_len = struct.unpack('>H', self.data[self.c+2:self.c+4])[0]
                        if len(self.data) - self.c >= 2+ll+mm+ws_len:
                            payload = self.data[self.c+2+ll+mm:self.c+2+ll+mm+ws_len]
                            if mask_flag == 0:
                                masks = b'\x00\x00\x00\x00'
                            else:
                                masks = self.data[self.c+2+ll:self.c+2+ll+4]
                            if mask_flag: 
                                payload = unmask_payload(payload, masks)
                            if ws_op == 0x82:
                                ret.append((payload, mask_flag))
                            found_frame = True
                            self.c += 2+ll+mm+ws_len
                            self.data = self.data[self.c:]
                            self.c = 0
                    if not found_frame:
                        break
        else:
            if pkt[TCP].flags & 3 : # SYN or FIN
                self.nseq = pkt[TCP].seq  + 1
        return ret

class GameConnection():
    def get_message(self, timeout=None):
        return None
    def send_message(self, msg):
        pass
    def close(self):
        pass

class PcapGameConnection(GameConnection):
    def __init__(self, filename=None, live=False):
        self.msgs = []
        self.to = None
        self.send_buf = b''
        if filename:
            self.pkts = []
            def dp(pkt):
                self.pkts.append(pkt)
            sniff(prn=dp, offline=filename)
#             print(self.pkts)
            self.pkts = iter(self.pkts)
        if live:
            self.sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.sk.bind(('0.0.0.0', 8989))
        def message_gen():
            ss = {}
#             n = 0
            while True:
#                 print('n -> ', n)
                if self.msgs:
                    yield self.msgs[0]
                    del self.msgs[0]
                else:
                    if filename and not live:
                        p = next(self.pkts)
                    if live:
                        try:
                            self.sk.settimeout(self.to)
                            data, addr = self.sk.recvfrom(8192)
                        except socket.timeout:
                            yield None, 0
                        except:
                            self.sk = None
                        p = Ether(data)
                    if (p[TCP].sport, p[TCP].dport) not in ss:
                        s = TS(p)
                        ss[(p[TCP].sport, p[TCP].dport)] = s
                    else:
                        s = ss[(p[TCP].sport, p[TCP].dport)]
                        ms = s.add_packet(p)
                        self.msgs += ms
#                 n += 1
        self.msg_gen = message_gen()
    def deal_pkt(self):
        pass
    def get_message(self, timeout=None):
        self.to = timeout
        msg, mask = next(self.msg_gen)
        if mask:
            print('Input MSG --------------------------------------------------')
        return msg

    def get_message_2(self, timeout=None):
        self.to = timeout
        while True:
            msg, mask = next(self.msg_gen)
            if msg == None:    ## when live mode and no notify
                return None
            if not timeout:
                if mask == 0:
                    return msg
                else:
                    if type(PKT.decode(msg)) == HEAT_BEAT: continue
                    if msg != self.send_buf:
                        cprint('################### send payload mismatch ######################', 'yellow')
                        print(msg)
                        print(self.send_buf)
                        print(PKT.decode(msg))
                        print('----------------------------------------------------------------')
                        return msg
#                         print(PKT.decode(self.send_buf))
                    else:
                        print('################### send payload matched ######################')
                        print(PKT.decode(self.send_buf))
                        print('***************************************************************')
            else:
                if type(PKT.decode(msg)) == SUB_S_OPERATE_NOTIFY:
                    return msg
                else:
                    self.msgs.insert(0, msg)
                    return None
                    
    def send_message(self, buf):
        if type(PKT.decode(buf)) == HEAT_BEAT: return
        self.send_buf = buf


In [3]:
class SUB_S_CHANGE_CANNON(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_CHANGE_CANNON']
    _t = 'MAIN_GS_GAME|SUB_S_CHANGE_CANNON'
    def decode_body(self, buf):
        self.cbChairID, self.cbStyle, self.wCost = struct.unpack('<BBh', buf)
PKT.register(SUB_S_CHANGE_CANNON)

class SUB_C_CHANGE_CANNON(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_CHANGE_CANNON']
    _t = 'MAIN_GS_GAME|SUB_C_CHANGE_CANNON'
    def decode_body(self, buf):
        self.cbStyle, self.wCost = struct.unpack('<Bh', buf)
PKT.register(SUB_C_CHANGE_CANNON)

class SUB_S_FIRE_SUCCESS(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_FIRE_SUCCESS']
    _t = 'MAIN_GS_GAME|SUB_S_FIRE_SUCCESS'
    def decode_body(self, buf):
        self.cbChairID, self.fRote, self.time = struct.unpack('<Bii', buf)
PKT.register(SUB_S_FIRE_SUCCESS)

class SUB_S_SEND_TIME(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_SEND_TIME']
    _t = 'MAIN_GS_GAME|SUB_S_SEND_TIME'
    def decode_body(self, buf):
        self.ulTime, = struct.unpack('<I', buf)
PKT.register(SUB_S_SEND_TIME)

class SUB_C_BUY_BULLET(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_BUY_BULLET']
    _t = 'MAIN_GS_GAME|SUB_C_BUY_BULLET'
    def decode_body(self, buf):
        self.dwCount, = struct.unpack('<I', buf)
PKT.register(SUB_C_BUY_BULLET)

class SUB_S_BUY_BULLET_SUCCESS(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_BUY_BULLET_SUCCESS']
    _t = 'MAIN_GS_GAME|SUB_S_BUY_BULLET_SUCCESS'
    def decode_body(self, buf):
        self.cbChairID, self.dwCount = struct.unpack('<BI', buf)
PKT.register(SUB_S_BUY_BULLET_SUCCESS)

class SUB_S_REWORD_NOTICX(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_REWORD_NOTICX']
    _t = 'MAIN_GS_GAME|SUB_S_REWORD_NOTICX'
    def decode_body(self, buf):
        self.cbChairID, self.cbType, self.dwValue = struct.unpack('<BBI', buf[:6])
        self.szName, n = read_str(buf[6:])
PKT.register(SUB_S_REWORD_NOTICX)

class SUB_C_SEND_TIME(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_SEND_TIME']
    _t = 'MAIN_GS_GAME|SUB_C_SEND_TIME'
    def decode_body(self, buf):
        self.ulTime, = struct.unpack('<I', buf)
PKT.register(SUB_C_SEND_TIME)

class SUB_C_FIRE(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_FIRE']
    _t = 'MAIN_GS_GAME|SUB_C_FIRE'
    def decode_body(self, buf):
        self.fRote, self.time = struct.unpack('<ii', buf)
PKT.register(SUB_C_FIRE)

class SUB_C_CAST_NET(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_CAST_NET']
    _t = 'MAIN_GS_GAME|SUB_C_CAST_NET'
    def decode_body(self, buf):
        assert len(buf)%3 == 1
        self.cbCount = buf[0]
        self.nets = []
        for i in range(self.cbCount):
            wID, cbType = struct.unpack('<HB', buf[1+3*i:4+3*i])
            self.nets.append({'wID':wID, 'cbType':cbType})
PKT.register(SUB_C_CAST_NET)

class SUB_C_HIT_TEAM(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_C_HIT_TEAM']
    _t = 'MAIN_GS_GAME|SUB_C_HIT_TEAM'
    def decode_body(self, buf):
#         assert len(buf)%3 == 1
        self.cbCount = buf[0]
        self.cbCount += 1
        self.nets = []
        for i in range(self.cbCount):
            wID, cbType = struct.unpack('<HB', buf[1+3*i:4+3*i])
            self.nets.append({'wID':wID, 'cbType':cbType})
PKT.register(SUB_C_HIT_TEAM)

class SUB_S_CAST_NET_SUCCESS(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_CAST_NET_SUCCESS']
    _t = 'MAIN_GS_GAME|SUB_S_CAST_NET_SUCCESS'
    def decode_body(self, buf):
        assert len(buf)%3 == 2
        self.cbCount, self.cbChairID, self.dwPlayerScore, self.dwPlayerAddScore, self.dwPlayerAddMedal = struct.unpack('<BBIII', buf[:14])
        self.nets = []
        for i in range(self.cbCount):
            wID, cbType = struct.unpack('<HB', buf[14+3*i:17+3*i])
            self.nets.append({'wID':wID, 'cbType':cbType})
PKT.register(SUB_S_CAST_NET_SUCCESS)

class SUB_S_SEND_FISH_PATH(PKT):
    main = codes_c2n['MAIN_GS_GAME']
    sub = codes_c2n['SUB_S_SEND_FISH_PATH']
    _t = 'MAIN_GS_GAME|SUB_S_SEND_FISH_PATH'
    def decode_body(self, buf):
        assert len(buf)%3 == 0
        self.cbType, self.cbCount, self.cbSeed = struct.unpack('<BBB', buf[:3])
        self.nets = []
        for i in range(self.cbCount):
            wID, cbType = struct.unpack('<HB', buf[3+3*i:6+3*i])
            self.nets.append({'wID':wID, 'cbType':cbType})
PKT.register(SUB_S_SEND_FISH_PATH)



In [22]:
conn = PcapGameConnection('/tmp/by.pcap')

In [8]:
conn = PcapGameConnection('/tmp/by_2.pcap')
n = 0
import traceback
while True:
    n += 1
    try:
#         if n > 400:
#             break
        try:
            msg = conn.get_message()
        except:
            break
        print('********************', n, msg)
        if not msg:
            break
        pkt = PKT.decode(msg)
    except:
        traceback.print_exc()
#         PKT.decode(msg)
        print('oops########')
        pass
    print(pkt)

&&&&&&&&&&&&&&&& 1555294953.430033
&&&&&&&&&&&&&&&& 1555294953.435635
&&&&&&&&&&&&&&&& 1555294953.456006
&&&&&&&&&&&&&&&& 1555294953.458622
&&&&&&&&&&&&&&&& 1555294953.468413
Input MSG --------------------------------------------------
******************** 1 b'2\x00\x00E\x100\x120\xb0\x00g\x03\x0093454a9844030803d794653ecd9a8167\x00\x03\x00\x00\x00'
MAIN_GS_LOGON|SUB_GS_LOGON_USERID ==> {'body_len': 50, 'id': 0, 'password': ('93454a9844030803d794653ecd9a8167', 32), 'pwd': '', 'userID': 57082032}
&&&&&&&&&&&&&&&& 1555294953.526534
******************** 2 b'\x08\x00\x00E\x100\x130'
MAIN_GS_LOGON|SUB_GS_LOGON_SUCCESS ==> {'body_len': 8}
&&&&&&&&&&&&&&&& 1555294953.561548
&&&&&&&&&&&&&&&& 1555294953.569364
******************** 3 b'\x12\x00\x01EP0Q0\t4\x85\x00B\x00\x1e\x00\x04\x00'
MAIN_GS_INFO|SUB_GS_SERVER_INFO ==> {'body_len': 18, 'chairCount': 4, 'kindID': 133, 'serverID': 13321, 'serverType': 66, 'tableCount': 30}
******************** 4 b"\x97\x00\x02E 0%0\x01\x00\x0e\x00\x00\x00\x00\x0

HEAT_BEAT|HEAT_BEAT ==> {'body_len': 16, 'raw': b'=\xeb\xb3\\\x00\x00\x00\x00'}
&&&&&&&&&&&&&&&& 1555295037.082298
&&&&&&&&&&&&&&&& 1555295038.333279
******************** 158 b'8\x00eE\xa00g\x00\x01\x0f\x0e\n\x00\x01-\x00\x011\x00\x012\x00\x033\x00\x034\x00\x045\x00\x056\x00\x067\x00\x078\x00\x089\x00\t:\x00\n;\x00\x11<\x00\x13=\x00\x14'
MAIN_GS_GAME|SUB_S_SEND_FISH_PATH ==> {'body_len': 56, 'cbCount': 15, 'cbSeed': 14, 'cbType': 1, 'nets': [{'wID': 10, 'cbType': 1}, {'wID': 45, 'cbType': 1}, {'wID': 49, 'cbType': 1}, {'wID': 50, 'cbType': 3}, {'wID': 51, 'cbType': 3}, {'wID': 52, 'cbType': 4}, {'wID': 53, 'cbType': 5}, {'wID': 54, 'cbType': 6}, {'wID': 55, 'cbType': 7}, {'wID': 56, 'cbType': 8}, {'wID': 57, 'cbType': 9}, {'wID': 58, 'cbType': 10}, {'wID': 59, 'cbType': 17}, {'wID': 60, 'cbType': 19}, {'wID': 61, 'cbType': 20}]}
&&&&&&&&&&&&&&&& 1555295038.427684
Input MSG --------------------------------------------------
******************** 159 b'\x10\x00\x00E\xa00\xb4\x00k\xc7\xff\

&&&&&&&&&&&&&&&& 1555295115.177954
&&&&&&&&&&&&&&&& 1555295115.189305
Input MSG --------------------------------------------------
******************** 292 b'\x10\x00\x00E\x00\x00\x01\x00\x8a\xeb\xb3\\\x00\x00\x00\x00'
HEAT_BEAT|HEAT_BEAT ==> {'body_len': 16, 'raw': b'\x8a\xeb\xb3\\\x00\x00\x00\x00'}
&&&&&&&&&&&&&&&& 1555295115.269655
&&&&&&&&&&&&&&&& 1555295115.285741
******************** 293 b'\x11\x00\xb8E\xa00g\x00\x04\x02UF\x00\x15I\x00\x17'
MAIN_GS_GAME|SUB_S_SEND_FISH_PATH ==> {'body_len': 17, 'cbCount': 2, 'cbSeed': 85, 'cbType': 4, 'nets': [{'wID': 70, 'cbType': 21}, {'wID': 73, 'cbType': 23}]}
&&&&&&&&&&&&&&&& 1555295115.293367
******************** 294 b'\x14\x00\xb9E\xa00g\x00\x05\x03\xd1J\x00\x01K\x00\x01L\x00\x01'
MAIN_GS_GAME|SUB_S_SEND_FISH_PATH ==> {'body_len': 20, 'cbCount': 3, 'cbSeed': 209, 'cbType': 5, 'nets': [{'wID': 74, 'cbType': 1}, {'wID': 75, 'cbType': 1}, {'wID': 76, 'cbType': 1}]}
******************** 295 b'\x14\x00\xbaE\xa00g\x00\x06\x03\xdaM\x00\x05N\x00\x

/usr/lib/python3/dist-packages/ipykernel_launcher.py:124: DeprecationWarning: generator 'PcapGameConnection.__init__.<locals>.message_gen' raised StopIteration


In [45]:
codes.MAIN_GS_LOGON

AttributeError: 'dict' object has no attribute 'MAIN_GS_LOGON'

In [37]:
conn.get_message()

KeyError: 193

In [41]:
%debug

> /root/py/codes.py(654)decode()
    652             msg.main = cmd_main
    653             msg.sub = cmd_sub
--> 654             msg._t = '%s|%s'%(codes[cmd_main],codes[cmd_sub])
    655         msg.body_len = len(buf)
    656         return msg

ipdb> cmd_main
12448
ipdb> quit
